In [1]:
# Imports
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.metrics import roc_auc_score

from fastai.vision.all import *
import timm
import torch
import time
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit

/student/sidd1091/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
csv_path = "/student/csc490_project/shared/labels.csv"
image_dir = "/student/csc490_project/shared/preprocessed_images/preprocessed_images"

df = pd.read_csv(csv_path)

# Keep only rows where 'Image Index' is in preprocessed images
preprocessed_images = set(os.listdir(image_dir))
df = df[df["Image Index"].isin(preprocessed_images)]
print(f"Filtered dataset contains {len(df)} images.")

# Convert 'Finding Labels' into a list of diseases
df["Finding Labels"] = df["Finding Labels"].apply(lambda x: x.split('|'))

# Convert labels into a binary multi-label format
mlb = MultiLabelBinarizer()
labels = pd.DataFrame(mlb.fit_transform(df["Finding Labels"]), columns=mlb.classes_)

# Merge binary labels with the dataset and keep 'Image Index', 'Patient ID', and label columns
df = df.join(labels)
df = df[['Image Index', 'Patient ID'] + list(mlb.classes_)]
print(f"Initial dataset shape: {df.shape}")

# Remove rare diseases with fewer than 2 samples (note: labels start at column index 2 now)
class_counts = df.iloc[:, 2:].sum()
rare_classes = class_counts[class_counts < 2].index.tolist()
print(f"Rare classes with <2 samples: {rare_classes}")
df = df.drop(columns=rare_classes)

# Remove rows where all labels are 0 (images that only had rare labels)
df = df[df.iloc[:, 2:].sum(axis=1) > 0]
print(f"Updated dataset shape after removing rare classes: {df.shape}")

# Split the dataset into training and test sets ensuring no patient appears in both sets
gss = GroupShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, test_idx = next(gss.split(df, groups=df["Patient ID"]))
print(f"Training set contains {len(train_idx)} images.")
print(f"Test set contains {len(test_idx)} images.")
train_df = df.iloc[train_idx]
test_df = df.iloc[test_idx]
# Verify that there is no overlap in patient IDs between training and test sets
print(f"Training set contains {len(train_df)} images from {train_df['Patient ID'].nunique()} unique patients.")
print(f"Test set contains {len(test_df)} images from {test_df['Patient ID'].nunique()} unique patients.")

common_patients = set(train_df["Patient ID"]).intersection(set(test_df["Patient ID"]))
assert len(common_patients) == 0, "OVERLAP OF PATIENTS BETWEEN TRAINING AND TEST SETS"

# Define disease labels (all columns except 'Image Index' and 'Patient ID')
disease_labels = list(train_df.columns[2:])

Filtered dataset contains 112120 images.
Initial dataset shape: (112120, 17)
Rare classes with <2 samples: []
Updated dataset shape after removing rare classes: (112120, 17)
Training set contains 78566 images.
Test set contains 33554 images.
Training set contains 78566 images from 21563 unique patients.
Test set contains 33554 images from 9242 unique patients.


In [3]:
def get_x(row): 
    return os.path.join(image_dir, row['Image Index'])

def get_y(row):
    return [label for label, value in zip(disease_labels, row[disease_labels]) if value == 1]

In [4]:
# Custom Transform for Gamma Correction
class GammaCorrection(Transform):
    def __init__(self, gamma:float=1.0):
        self.gamma = gamma
    def encodes(self, img:PILImage):
        # Expecting a grayscale PIL image.
        img_np = np.array(img).astype(np.float32) / 255.0
        corrected = np.power(img_np, self.gamma)
        corrected = np.clip(corrected * 255, 0, 255).astype(np.uint8)
        return PILImage.create(corrected)

# Transform to convert image to 3 channels
def to_3channel(img:PILImage):
    return img.convert("RGB")

In [5]:
# Create a DataBlock for multi-label classification.
dblock = DataBlock(
    blocks=(ImageBlock, MultiCategoryBlock(vocab=disease_labels)),
    get_x=get_x,
    get_y=get_y,
    splitter=IndexSplitter(test_idx),
    item_tfms=[GammaCorrection(gamma=0.8), to_3channel, Resize(224)],
    batch_tfms=[*aug_transforms(flip_vert=False, max_rotate=15, max_zoom=1.0, max_warp=0.),
                Normalize.from_stats([0.485,0.456,0.406],[0.229,0.224,0.225])]
)

# Create DataLoaders from the full DataFrame
dls = dblock.dataloaders(df, bs=16)

In [6]:
model_name = 'maxvit_rmlp_base_rw_224.sw_in12k_ft_in1k'
num_classes = len(disease_labels)  # This should match the number of labels
model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss_func = BCEWithLogitsLossFlat()

In [ ]:
learn = Learner(
    dls,
    model,
    loss_func=loss_func,
    metrics=[
        AccumMetric(accuracy_multi, name='acc_multi'),  
        F1ScoreMulti(),                                 
        RocAucMulti()                                   
    ]
)

learn.fit_one_cycle(
    20,
    lr_max=1e-4,
    cbs=[
        SaveModelCallback(monitor='roc_auc_score', fname='noverlap_maxvit'),
        EarlyStoppingCallback(monitor='roc_auc_score', patience=3)
    ]
)


epoch,train_loss,valid_loss,acc_multi,f1_score,roc_auc_score,time
0,0.188939,0.188622,0.931656,0.088319,0.777953,21:47
1,0.177558,0.178244,0.934092,0.160830,0.813146,21:48
2,0.175489,0.179330,0.933093,0.151531,0.819506,21:50
3,0.167723,0.176386,0.934516,0.164062,0.828106,21:51
4,0.170144,0.177603,0.933210,0.222497,0.827747,21:50
5,0.155536,0.177014,0.932894,0.244474,0.833594,21:51
6,0.160357,0.177115,0.933818,0.236241,0.833638,21:52
7,0.146920,0.181853,0.931972,0.234302,0.826872,21:52
8,0.138571,0.190051,0.930053,0.251181,0.814958,21:50
9,0.129253,0.198422,0.927875,0.282155,0.810861,21:51


Better model found at epoch 0 with roc_auc_score value: 0.7779528438245927.
Better model found at epoch 1 with roc_auc_score value: 0.8131464287645694.
Better model found at epoch 2 with roc_auc_score value: 0.8195055324277573.
Better model found at epoch 3 with roc_auc_score value: 0.8281061206421545.
Better model found at epoch 5 with roc_auc_score value: 0.8335943863157624.
Better model found at epoch 6 with roc_auc_score value: 0.833637727729217.
No improvement since epoch 6: early stopping


/student/sidd1091/.local/lib/python3.11/site-packages/fastai/learner.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device, **tor

: 